In [1]:
!pip install pandas


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import torch

print("PyTorch version:", torch.__version__)


PyTorch version: 2.1.0+cu118


In [3]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
import torchvision.models as models
import time

In [4]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
print(device)

cuda


In [6]:
# Data augmentation for training
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomCrop(32, padding=4),  # Randomly crop the image
    transforms.Resize(224),  # Resize to 224x224 for ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# No augmentation for validation and test
val_test_transform = transforms.Compose([
    transforms.Resize(224),  # Resize to 224x224 for ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load CIFAR-10 dataset
train_val_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=train_transform)
test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=val_test_transform)

# Split train_val_dataset into train and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(train_val_dataset))
val_size = len(train_val_dataset) - train_size
train_dataset, val_dataset = random_split(train_val_dataset, [train_size, val_size])

# Apply val_test_transform to the validation set
val_dataset.dataset.transform = val_test_transform

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

58.1%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

100.0%


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
# Load pretrained ResNet-50 (Teacher Model)
teacher = models.resnet50(pretrained=True)

# Modify the final fully connected layer for 10 classes (CIFAR-10)
teacher.fc = nn.Linear(teacher.fc.in_features, 100)
# Move models to device
teacher = teacher.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:

model_path = 'best_teacher_model.pth'
# Load the model weights
teacher.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [23]:
teacher_accuracy = evaluate(teacher, test_loader, device)
print(f"Teacher Model Test Accuracy: {teacher_accuracy:.2f}%")

Teacher Model Test Accuracy: 80.89%


In [10]:
# Load pretrained ResNet-18 (Student Model)
student = models.resnet18(pretrained=True)
# Modify the final fully connected layer for 10 classes (CIFAR-10)
student.fc = nn.Linear(student.fc.in_features, 100)
student = student.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [27]:

# model_path = 'student_before_pruning.pth'
# # Load the model weights
# student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

In [11]:
# Logits normalization function
def normalize(logit):
    mean = logit.mean(dim=-1, keepdim=True)
    stdv = logit.std(dim=-1, keepdim=True)
    return (logit - mean) / (1e-7 + stdv)


In [12]:
# CA-KLD Loss for Classification
def cakld_loss(student_logits, teacher_logits, beta_prob):
    # Forward KL (student || teacher)
    student_log_prob = F.log_softmax(student_logits, dim=1)
    teacher_prob = F.softmax(teacher_logits, dim=1)
    forward_kl = F.kl_div(student_log_prob, teacher_prob, reduction='batchmean')

    # Reverse KL (teacher || student)
    teacher_log_prob = F.log_softmax(teacher_logits, dim=1)
    student_prob = F.softmax(student_logits, dim=1)
    reverse_kl = F.kl_div(teacher_log_prob, student_prob, reduction='batchmean')

    # Combined KL loss
    kl_loss = beta_prob * reverse_kl + (1 - beta_prob) * forward_kl
    return kl_loss


In [13]:
def evaluate(model, test_loader, device):
    model = model.to(device)  # Ensure model is on the correct device
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total


In [14]:
def calculate_sparsity(model):
    total_zeros = 0
    total_params = 0
    for name, param in model.named_parameters():
        if 'weight' in name:
            total_zeros += torch.sum(param == 0).item()
            total_params += param.numel()
    return total_zeros / total_params

In [15]:
import torch
import time
def measure_inference_time(model, test_loader, num_runs=5):
    device = torch.device('cpu')
    model.eval()
    model.to(device)

    # Warm-up (one batch to avoid startup cost)
    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            _ = model(inputs)
            break

    total_time = 0
    total_images = 0

    with torch.no_grad():
        for _ in range(num_runs):
            for inputs, _ in test_loader:
                inputs = inputs.to(device)
                batch_size = inputs.size(0)
                start_time = time.time()
                _ = model(inputs)
                end_time = time.time()

                total_time += (end_time - start_time)
                total_images += batch_size

    avg_time_per_image = total_time / total_images
    return avg_time_per_image


In [16]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

def calculate_model_size(model, filename="temp.pth"):
    torch.save(model.state_dict(), filename)
    size = os.path.getsize(filename) / (1024 * 1024)  # Size in MB
    os.remove(filename)
    return size

def compare_model_sizes(teacher, student, pruned_student):
    # Count parameters
    teacher_params = count_parameters(teacher)
    student_params = count_parameters(student)
    pruned_params = count_parameters(pruned_student)
    
    # Calculate disk size
    teacher_size = calculate_model_size(teacher, "teacher.pth")
    student_size = calculate_model_size(student, "student.pth")
    pruned_size = calculate_model_size(pruned_student, "pruned_student.pth")
    
    # Print comparison
    print("\n--- Model Size Comparison ---")
    print(f"Teacher Model: {teacher_params} parameters, {teacher_size:.2f} MB")
    print(f"Student Model (Before Pruning): {student_params} parameters, {student_size:.2f} MB")
    print(f"Student Model (After Pruning): {pruned_params} parameters, {pruned_size:.2f} MB")
    
    # Calculate compression ratio
    compression_ratio = student_size / pruned_size
    print(f"\nCompression Ratio: {compression_ratio:.2f}x")

In [17]:
def train_model(model, train_loader, val_loader, epochs=10, lr=0.001, patience=3):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    
    best_val_accuracy = 0.0
    best_model_state = None
    patience_counter = 0  # Counter for early stopping
    
    for epoch in range(epochs):
        print(epoch)
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Evaluate on the validation set
        val_accuracy = evaluate(model, val_loader, device)
        print(f"Epoch {epoch+1}/{epochs} | Loss: {running_loss/len(train_loader):.4f} | Val Accuracy: {val_accuracy:.2f}%")
        
        # Early stopping logic
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_state = model.state_dict()
            patience_counter = 0  # Reset patience counter
            torch.save(model.state_dict(), 'best_teacher_model.pth')  # Save the best model
            print(f" New best model saved with validation accuracy: {best_val_accuracy:.2f}%")
        else:
            patience_counter += 1
            print(f" No improvement in validation accuracy ({patience_counter}/{patience})")
            
            # Stop training if no improvement for 'patience' epochs
            if patience_counter >= patience:
                print(f"\nEarly stopping triggered! No improvement for {patience} epochs.")
                break
    
    # Load the best model state
    model.load_state_dict(torch.load('best_teacher_model.pth'))
    print("\nLoading the best model for final evaluation.")
    
    # Evaluate on the test set
    test_accuracy = evaluate(model, test_loader, device)
    print(f"Test Accuracy with Best Model: {test_accuracy:.2f}%")
    
    return model



In [18]:
def compute_gradient_importance(
    teacher, student, data_loader, device, temperature=4.0, alpha=0.5, beta_prob=0.5, accumulation_epochs=3
):
    importance_scores = {}

    # Initialize importance score storage for conv layer weights only
    for name, param in student.named_parameters():
        if 'weight' in name and len(param.shape) == 4:  # Conv weights only
            importance_scores[name] = torch.zeros_like(param.data, device=device)

    teacher.to(device).eval()
    student.to(device).train()

    # Add momentum for gradient accumulation smoothing
    momentum = 0.9  # Controls exponential moving average
    accumulated_batches = 0  # Track for bias correction

    for epoch in range(accumulation_epochs):
        print(f"Accumulation Epoch {epoch+1}/{accumulation_epochs}")
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            student.zero_grad()

            with torch.no_grad():
                teacher_logits = teacher(inputs)

            student_logits = student(inputs)

            # Temperature scaling
            student_logits_temp = student_logits / temperature
            teacher_logits_temp = teacher_logits / temperature

            # Compute losses
            distillation_loss = cakld_loss(student_logits_temp, teacher_logits_temp, beta_prob) * (temperature ** 2)
            ce_loss = F.cross_entropy(student_logits, labels)
            loss = alpha * distillation_loss + (1 - alpha) * ce_loss

            # Modified backward propagation
            loss.backward()

            # Accumulate importance scores with parameter-gradient product
            accumulated_batches += 1
            for name, param in student.named_parameters():
                if name in importance_scores and param.grad is not None:
                    # Key modification: Use parameter-gradient product magnitude
                    grad_product = (param.data * param.grad).abs_()
                    
                    # Exponential moving average with bias correction
                    if accumulated_batches == 1:
                        importance_scores[name] = grad_product
                    else:
                        importance_scores[name] = momentum * importance_scores[name] + (1 - momentum) * grad_product

    # Apply bias correction for EMA
    for name in importance_scores:
        importance_scores[name] /= (1 - momentum**accumulated_batches)

    return importance_scores

In [19]:
def gradient_based_global_prune(model, importance_scores, prune_ratio=0.95):
    all_scores = torch.cat([score.flatten() for score in importance_scores.values()])
    threshold = torch.topk(all_scores, k=int(prune_ratio * all_scores.numel()), largest=False)[0][-1]

    for name, param in model.named_parameters():
        if name in importance_scores:
            mask = (importance_scores[name] > threshold).float()
            param.data.mul_(mask)

    return model


In [20]:
import torch
import torch.nn.functional as F
import torch.optim as optim

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

def retrain_with_sparsity(student, train_loader, val_loader, epochs=5, save_path="retrained_student_model.pt", patience=3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

    # 1. Store masks AND zero momentum buffers for pruned weights
    masks = {}
    for name, param in student.named_parameters():
        if 'weight' in name and param.dim() == 4:  # Consider only conv layers
            mask = (param != 0).float().to(device)
            masks[name] = mask
            # Zero momentum buffers for pruned weights
            if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                optimizer.state[param]['momentum_buffer'] *= mask

    student = student.to(device)
    best_val_acc = 0.0
    best_model = None
    patience_counter = 0  # Counter for early stopping

    # 2. Add gradient clipping to prevent NaN
    max_grad_norm = 1.0

    for epoch in range(epochs):
        student.train()
        total_loss = 0.0
        correct, total = 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = student(inputs)
            loss = F.cross_entropy(outputs, labels)
            loss.backward()

            # Apply masks to gradients
            for name, param in student.named_parameters():
                if name in masks:
                    param.grad.data *= masks[name]

            # Gradient clipping before optimizer step
            torch.nn.utils.clip_grad_norm_(student.parameters(), max_grad_norm)

            optimizer.step()

            # Reapply masks and update momentum buffers
            for name, param in student.named_parameters():
                if name in masks:
                    param.data *= masks[name]
                    if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                        optimizer.state[param]['momentum_buffer'] *= masks[name]

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Validation phase
        student.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = student(inputs)
                loss = F.cross_entropy(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = 100.0 * val_correct / val_total

        # Track best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model = student.state_dict()
            torch.save(best_model, save_path)
            patience_counter = 0  # Reset patience counter
            print(f"New best model saved with Val Accuracy: {best_val_acc:.2f}%")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break  # Stop training

        # Print results
        sparsity = calculate_sparsity(student)
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"Validation Loss: {val_loss:.4f} | Validation Acc: {val_acc:.2f}% | Sparsity: {sparsity*100:.2f}%\n")

    print(f"Best Validation Accuracy: {best_val_acc:.2f}% | Best Model Saved at: {save_path}")
    return student

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time

# KD training with CA-KLD loss and mask-based momentum handling
def retrain_with_KD(teacher, student, train_loader, val_loader, epochs=50,
                    temperature=5.0, alpha=0.5, beta_prob=0.5, patience=5,
                    save_path="student_before_pruning.pth"):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

    # 1. Store masks and zero momentum buffers
    masks = {}
    for name, param in student.named_parameters():
        if 'weight' in name and param.dim() == 4:
            mask = (param != 0).float().to(device)
            masks[name] = mask
            if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                optimizer.state[param]['momentum_buffer'] *= mask

    teacher = teacher.to(device).eval()
    student = student.to(device)

    best_val_acc = 0.0
    best_model_state = None
    patience_counter = 0
    start_time = time.time()

    for epoch in range(epochs):
        student.train()
        total_loss, correct, total = 0.0, 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with torch.no_grad():
                teacher_logits = teacher(inputs)

            student_logits = student(inputs)

            # Apply temperature
            teacher_logits_temp = teacher_logits / temperature
            student_logits_temp = student_logits / temperature

            # Logits normalization
            teacher_logits_temp = normalize(teacher_logits_temp)
            student_logits_temp = normalize(student_logits_temp)


            # CA-KLD loss
            kd_loss = cakld_loss(student_logits_temp, teacher_logits_temp, beta_prob) * (temperature ** 2)
            ce_loss = F.cross_entropy(student_logits, labels)

            loss = alpha * kd_loss + (1 - alpha) * ce_loss
            loss.backward()
            optimizer.step()

            # Reapply masks and update momentum
            for name, param in student.named_parameters():
                if name in masks:
                    param.data *= masks[name]
                    if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                        optimizer.state[param]['momentum_buffer'] *= masks[name]

            total_loss += loss.item()
            _, predicted = student_logits.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Validation
        student.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = student(inputs)
                loss = F.cross_entropy(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = 100.0 * val_correct / val_total
        sparsity = calculate_sparsity(student) * 100.0  # Assuming this function is defined elsewhere

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | Sparsity: {sparsity:.2f}%")

        # Early stopping logic
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = student.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break

    # Restore and save best model
    student.load_state_dict(best_model_state)
    torch.save(student.state_dict(), save_path)
    print(f"Student model saved before pruning at: {save_path}")
    total_time = time.time() - start_time
    print(f"Total Training Time: {total_time // 60:.0f}m {total_time % 60:.0f}s")

    return student

In [22]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

# Training function with KD + CA-KLD and logits normalization
def train_kd_pruning(teacher, student, train_loader, val_loader, epochs=50, temperature=5.0, alpha=0.5,
                     beta_prob=0.5, patience=5, save_path="student_before_pruning.pth"):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

    teacher = teacher.to(device)
    student = student.to(device)
    teacher.eval()  # Freeze teacher

    best_val_acc = 0.0
    best_model_state = None
    patience_counter = 0
    start_time = time.time()

    for epoch in range(epochs):
        student.train()
        total_loss = 0.0
        correct, total = 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            with torch.no_grad():
                teacher_logits = teacher(inputs)

            student_logits = student(inputs)

            # Temperature scaling
            teacher_logits_temp = teacher_logits / temperature
            student_logits_temp = student_logits / temperature

            # Logits normalization
            teacher_logits_temp = normalize(teacher_logits_temp)
            student_logits_temp = normalize(student_logits_temp)

            # CA-KLD loss (normalized logits)
            distillation_loss = cakld_loss(student_logits_temp, teacher_logits_temp, beta_prob) * (temperature ** 2)

            # Cross-entropy loss
            ground_truth_loss = F.cross_entropy(student_logits, labels)

            # Combined loss
            loss = alpha * distillation_loss + (1 - alpha) * ground_truth_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = student_logits.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Validation accuracy
        val_acc = evaluate(student, val_loader, device)

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | "
              f"Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = student.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break

    # Load best model state and save
    student.load_state_dict(best_model_state)
    torch.save(student.state_dict(), save_path)
    print(f"Student model saved before pruning at: {save_path}")

    total_time = time.time() - start_time
    print(f"Total Training Time: {total_time // 60:.0f}m {total_time % 60:.0f}s")

    return student

In [40]:

student = train_kd_pruning(
    teacher, student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.5,beta_prob=0.5, patience=5,save_path="student_before_pruning.pth"
)


Epoch 1/50 | Train Loss: 9.5556 | Train Acc: 54.33% | Val Acc: 61.88%
Epoch 2/50 | Train Loss: 4.3486 | Train Acc: 74.08% | Val Acc: 67.95%
Epoch 3/50 | Train Loss: 2.7151 | Train Acc: 82.97% | Val Acc: 74.83%
Epoch 4/50 | Train Loss: 1.7075 | Train Acc: 88.86% | Val Acc: 77.60%
Epoch 5/50 | Train Loss: 1.1438 | Train Acc: 92.86% | Val Acc: 80.32%
Epoch 6/50 | Train Loss: 0.8663 | Train Acc: 94.71% | Val Acc: 80.73%
Epoch 7/50 | Train Loss: 0.7182 | Train Acc: 95.67% | Val Acc: 81.27%
Epoch 8/50 | Train Loss: 0.6282 | Train Acc: 96.04% | Val Acc: 81.15%
Epoch 9/50 | Train Loss: 0.5653 | Train Acc: 96.42% | Val Acc: 81.58%
Epoch 10/50 | Train Loss: 0.5261 | Train Acc: 96.51% | Val Acc: 81.73%
Epoch 11/50 | Train Loss: 0.4904 | Train Acc: 96.73% | Val Acc: 81.63%
Epoch 12/50 | Train Loss: 0.4693 | Train Acc: 96.81% | Val Acc: 81.57%
Epoch 13/50 | Train Loss: 0.4545 | Train Acc: 96.77% | Val Acc: 81.88%
Epoch 14/50 | Train Loss: 0.4359 | Train Acc: 96.93% | Val Acc: 82.24%
Epoch 15/50 | T

In [41]:
# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity Before Pruning: {sparsity * 100:.2f}%")

teacher_accuracy = evaluate(teacher, test_loader, device)
student_accuracy = evaluate(student, test_loader, device)
print(f"Teacher Model Test Accuracy: {teacher_accuracy:.2f}%")
print(f"Student Model Test Accuracy Before Pruning: {student_accuracy:.2f}%")

Sparsity Before Pruning: 0.00%
Teacher Model Test Accuracy: 80.89%
Student Model Test Accuracy Before Pruning: 81.12%


## 93% Sparsity

In [42]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [43]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.94)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 35s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [44]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 1.00%


In [45]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=200,  save_path='retrained_student_model.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 66.42%
Epoch 1/200 | Train Loss: 1.6213 | Train Acc: 59.18%
Validation Loss: 1.1940 | Validation Acc: 66.42% | Sparsity: 93.53%

New best model saved with Val Accuracy: 69.90%
Epoch 2/200 | Train Loss: 0.8153 | Train Acc: 76.36%
Validation Loss: 1.0408 | Validation Acc: 69.90% | Sparsity: 93.53%

New best model saved with Val Accuracy: 71.53%
Epoch 3/200 | Train Loss: 0.6268 | Train Acc: 81.58%
Validation Loss: 0.9868 | Validation Acc: 71.53% | Sparsity: 93.53%

New best model saved with Val Accuracy: 72.07%
Epoch 4/200 | Train Loss: 0.5024 | Train Acc: 85.32%
Validation Loss: 0.9658 | Validation Acc: 72.07% | Sparsity: 93.53%

New best model saved with Val Accuracy: 72.91%
Epoch 5/200 | Train Loss: 0.4129 | Train Acc: 88.17%
Validation Loss: 0.9567 | Validation Acc: 72.91% | Sparsity: 93.53%

Epoch 6/200 | Train Loss: 0.3364 | Train Acc: 90.86%
Validation Loss: 0.9595 | Validation Acc: 72.85% | Sparsity: 93.53%

New best model saved with Val Acc

In [46]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 72.00%


In [49]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [50]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.94)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 36s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [51]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 8.8002 | Train Acc: 64.75% | Val Loss: 1.4439 | Val Acc: 65.64% | Sparsity: 93.53%
Epoch 2/50 | Train Loss: 5.0399 | Train Acc: 77.30% | Val Loss: 1.3628 | Val Acc: 68.55% | Sparsity: 93.53%
Epoch 3/50 | Train Loss: 3.8382 | Train Acc: 82.15% | Val Loss: 1.1598 | Val Acc: 72.44% | Sparsity: 93.53%
Epoch 4/50 | Train Loss: 2.9677 | Train Acc: 86.17% | Val Loss: 1.0755 | Val Acc: 72.72% | Sparsity: 93.53%
Epoch 5/50 | Train Loss: 2.3849 | Train Acc: 88.56% | Val Loss: 1.0235 | Val Acc: 74.34% | Sparsity: 93.53%
Epoch 6/50 | Train Loss: 2.0129 | Train Acc: 90.52% | Val Loss: 1.0252 | Val Acc: 75.00% | Sparsity: 93.53%
Epoch 7/50 | Train Loss: 1.7389 | Train Acc: 91.72% | Val Loss: 0.9920 | Val Acc: 75.67% | Sparsity: 93.53%
Epoch 8/50 | Train Loss: 1.5394 | Train Acc: 92.58% | Val Loss: 0.9375 | Val Acc: 76.06% | Sparsity: 93.53%
Epoch 9/50 | Train Loss: 1.3975 | Train Acc: 93.35% | Val Loss: 0.9334 | Val Acc: 76.57% | Sparsity: 93.53%
Epoch 10/50 | Train Loss: 1.

In [52]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 76.35%


In [53]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [54]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=6.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.94)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 37s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [55]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=6.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 14.3468 | Train Acc: 60.67% | Val Loss: 2.3656 | Val Acc: 58.10% | Sparsity: 93.53%
Epoch 2/50 | Train Loss: 8.6533 | Train Acc: 73.26% | Val Loss: 1.7821 | Val Acc: 66.23% | Sparsity: 93.53%
Epoch 3/50 | Train Loss: 6.4335 | Train Acc: 79.29% | Val Loss: 1.4357 | Val Acc: 69.29% | Sparsity: 93.53%
Epoch 4/50 | Train Loss: 4.9439 | Train Acc: 83.54% | Val Loss: 1.2908 | Val Acc: 71.94% | Sparsity: 93.53%
Epoch 5/50 | Train Loss: 4.0082 | Train Acc: 86.58% | Val Loss: 1.1789 | Val Acc: 73.06% | Sparsity: 93.53%
Epoch 6/50 | Train Loss: 3.3317 | Train Acc: 88.74% | Val Loss: 1.1339 | Val Acc: 74.19% | Sparsity: 93.53%
Epoch 7/50 | Train Loss: 2.8698 | Train Acc: 90.24% | Val Loss: 1.1256 | Val Acc: 73.94% | Sparsity: 93.53%
Epoch 8/50 | Train Loss: 2.4823 | Train Acc: 91.63% | Val Loss: 1.0222 | Val Acc: 75.47% | Sparsity: 93.53%
Epoch 9/50 | Train Loss: 2.2157 | Train Acc: 92.42% | Val Loss: 1.0269 | Val Acc: 76.20% | Sparsity: 93.53%
Epoch 10/50 | Train Loss: 1

In [56]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 75.34%


# 98% of Sparsity

In [57]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [58]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.985)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 37s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [59]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=200,  save_path='retrained_student_model.pt',patience=7
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 26.68%
Epoch 1/200 | Train Loss: 3.9633 | Train Acc: 12.63%
Validation Loss: 3.1184 | Validation Acc: 26.68% | Sparsity: 98.01%

New best model saved with Val Accuracy: 45.58%
Epoch 2/200 | Train Loss: 2.4241 | Train Acc: 38.88%
Validation Loss: 2.0475 | Validation Acc: 45.58% | Sparsity: 98.01%

New best model saved with Val Accuracy: 52.40%
Epoch 3/200 | Train Loss: 1.8531 | Train Acc: 50.59%
Validation Loss: 1.7403 | Validation Acc: 52.40% | Sparsity: 98.01%

New best model saved with Val Accuracy: 55.84%
Epoch 4/200 | Train Loss: 1.5834 | Train Acc: 56.62%
Validation Loss: 1.5856 | Validation Acc: 55.84% | Sparsity: 98.01%

New best model saved with Val Accuracy: 58.28%
Epoch 5/200 | Train Loss: 1.4103 | Train Acc: 60.70%
Validation Loss: 1.4864 | Validation Acc: 58.28% | Sparsity: 98.01%

New best model saved with Val Accuracy: 59.38%
Epoch 6/200 | Train Loss: 1.2871 | Train Acc: 63.72%
Validation Loss: 1.4321 | Validation Acc: 59.38% | Spar

In [60]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 62.49%


In [61]:

# pruned_student_inference_time = measure_inference_time(retrained_student, test_loader,)
# print(f"Pruned Student Model Inference Time: {pruned_student_inference_time * 1000:.2f} ms per batch")


# Retrained with KD

In [65]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [66]:
student_accuracy = evaluate(student, test_loader, device)
print(f"Student Model Test Accuracy: {student_accuracy:.2f}%")

Student Model Test Accuracy: 81.12%


In [67]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.985)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 39s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [68]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=7,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 17.6078 | Train Acc: 39.85% | Val Loss: 2.0596 | Val Acc: 49.63% | Sparsity: 98.01%
Epoch 2/50 | Train Loss: 10.6520 | Train Acc: 58.28% | Val Loss: 1.6306 | Val Acc: 56.69% | Sparsity: 98.01%
Epoch 3/50 | Train Loss: 8.9145 | Train Acc: 63.92% | Val Loss: 1.5456 | Val Acc: 59.53% | Sparsity: 98.01%
Epoch 4/50 | Train Loss: 7.9609 | Train Acc: 66.99% | Val Loss: 1.4486 | Val Acc: 61.66% | Sparsity: 98.01%
Epoch 5/50 | Train Loss: 7.1771 | Train Acc: 69.94% | Val Loss: 1.4311 | Val Acc: 63.29% | Sparsity: 98.01%
Epoch 6/50 | Train Loss: 6.5889 | Train Acc: 72.07% | Val Loss: 1.3731 | Val Acc: 64.01% | Sparsity: 98.01%
Epoch 7/50 | Train Loss: 6.1054 | Train Acc: 73.96% | Val Loss: 1.3315 | Val Acc: 65.31% | Sparsity: 98.01%
Epoch 8/50 | Train Loss: 5.7066 | Train Acc: 75.32% | Val Loss: 1.3650 | Val Acc: 65.15% | Sparsity: 98.01%
Epoch 9/50 | Train Loss: 5.4041 | Train Acc: 76.61% | Val Loss: 1.3407 | Val Acc: 64.96% | Sparsity: 98.01%
Epoch 10/50 | Train Loss: 

In [69]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 67.06%


In [70]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [71]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.985)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 38s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [72]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=7,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 7.1997 | Train Acc: 37.82% | Val Loss: 1.8237 | Val Acc: 52.52% | Sparsity: 98.01%
Epoch 2/50 | Train Loss: 4.0921 | Train Acc: 59.13% | Val Loss: 1.4799 | Val Acc: 59.35% | Sparsity: 98.01%
Epoch 3/50 | Train Loss: 3.3648 | Train Acc: 65.03% | Val Loss: 1.4344 | Val Acc: 60.89% | Sparsity: 98.01%
Epoch 4/50 | Train Loss: 2.9779 | Train Acc: 68.72% | Val Loss: 1.3388 | Val Acc: 63.46% | Sparsity: 98.01%
Epoch 5/50 | Train Loss: 2.6975 | Train Acc: 71.27% | Val Loss: 1.2737 | Val Acc: 65.56% | Sparsity: 98.01%
Epoch 6/50 | Train Loss: 2.4825 | Train Acc: 73.22% | Val Loss: 1.2823 | Val Acc: 64.76% | Sparsity: 98.01%
Epoch 7/50 | Train Loss: 2.3231 | Train Acc: 74.94% | Val Loss: 1.3071 | Val Acc: 64.63% | Sparsity: 98.01%
Epoch 8/50 | Train Loss: 2.1720 | Train Acc: 76.36% | Val Loss: 1.2874 | Val Acc: 65.63% | Sparsity: 98.01%
Epoch 9/50 | Train Loss: 2.0501 | Train Acc: 77.63% | Val Loss: 1.2614 | Val Acc: 65.95% | Sparsity: 98.01%
Epoch 10/50 | Train Loss: 1.

In [73]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 66.32%


In [26]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [27]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.9789)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 19s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [28]:
# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity Before Pruning: {sparsity * 100:.2f}%")

Sparsity Before Pruning: 97.40%


In [29]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=7,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 6.0904 | Train Acc: 45.49% | Val Loss: 1.5284 | Val Acc: 59.35% | Sparsity: 97.40%
Epoch 2/50 | Train Loss: 3.3372 | Train Acc: 65.29% | Val Loss: 1.3383 | Val Acc: 63.08% | Sparsity: 97.40%
Epoch 3/50 | Train Loss: 2.7229 | Train Acc: 70.53% | Val Loss: 1.2166 | Val Acc: 66.90% | Sparsity: 97.40%
Epoch 4/50 | Train Loss: 2.3848 | Train Acc: 73.81% | Val Loss: 1.1921 | Val Acc: 67.17% | Sparsity: 97.40%
Epoch 5/50 | Train Loss: 2.1276 | Train Acc: 76.63% | Val Loss: 1.1354 | Val Acc: 69.12% | Sparsity: 97.40%
Epoch 6/50 | Train Loss: 1.9232 | Train Acc: 78.67% | Val Loss: 1.0933 | Val Acc: 69.82% | Sparsity: 97.40%
Epoch 7/50 | Train Loss: 1.7790 | Train Acc: 80.48% | Val Loss: 1.1426 | Val Acc: 69.28% | Sparsity: 97.40%
Epoch 8/50 | Train Loss: 1.6374 | Train Acc: 81.93% | Val Loss: 1.0990 | Val Acc: 69.62% | Sparsity: 97.40%
Epoch 9/50 | Train Loss: 1.5176 | Train Acc: 83.29% | Val Loss: 1.1161 | Val Acc: 69.80% | Sparsity: 97.40%
Epoch 10/50 | Train Loss: 1.

In [30]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 68.33%


In [31]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [32]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.9789)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 22s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [33]:
# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity Before Pruning: {sparsity * 100:.2f}%")

Sparsity Before Pruning: 97.40%


In [34]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=7,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 15.5049 | Train Acc: 45.63% | Val Loss: 2.1400 | Val Acc: 51.76% | Sparsity: 97.40%
Epoch 2/50 | Train Loss: 9.1543 | Train Acc: 63.20% | Val Loss: 1.6100 | Val Acc: 60.35% | Sparsity: 97.40%
Epoch 3/50 | Train Loss: 7.6077 | Train Acc: 68.13% | Val Loss: 1.4608 | Val Acc: 64.41% | Sparsity: 97.40%
Epoch 4/50 | Train Loss: 6.6433 | Train Acc: 71.94% | Val Loss: 1.3390 | Val Acc: 65.79% | Sparsity: 97.40%
Epoch 5/50 | Train Loss: 5.9056 | Train Acc: 74.39% | Val Loss: 1.3043 | Val Acc: 66.40% | Sparsity: 97.40%
Epoch 6/50 | Train Loss: 5.3353 | Train Acc: 76.52% | Val Loss: 1.3862 | Val Acc: 65.83% | Sparsity: 97.40%
Epoch 7/50 | Train Loss: 4.8409 | Train Acc: 78.63% | Val Loss: 1.2509 | Val Acc: 68.31% | Sparsity: 97.40%
Epoch 8/50 | Train Loss: 4.4752 | Train Acc: 80.15% | Val Loss: 1.2567 | Val Acc: 67.87% | Sparsity: 97.40%
Epoch 9/50 | Train Loss: 4.1774 | Train Acc: 81.19% | Val Loss: 1.2216 | Val Acc: 68.08% | Sparsity: 97.40%
Epoch 10/50 | Train Loss: 3

In [35]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 68.66%


## 90% Sparsity

In [74]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [75]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.91)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 38s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [76]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=100,  save_path='retrained_student_model_80.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 72.30%
Epoch 1/100 | Train Loss: 1.0398 | Train Acc: 73.54%
Validation Loss: 0.9920 | Validation Acc: 72.30% | Sparsity: 90.55%

New best model saved with Val Accuracy: 74.38%
Epoch 2/100 | Train Loss: 0.5256 | Train Acc: 84.89%
Validation Loss: 0.9011 | Validation Acc: 74.38% | Sparsity: 90.55%

New best model saved with Val Accuracy: 74.70%
Epoch 3/100 | Train Loss: 0.3809 | Train Acc: 89.05%
Validation Loss: 0.8945 | Validation Acc: 74.70% | Sparsity: 90.55%

New best model saved with Val Accuracy: 74.90%
Epoch 4/100 | Train Loss: 0.2852 | Train Acc: 92.28%
Validation Loss: 0.9007 | Validation Acc: 74.90% | Sparsity: 90.55%

New best model saved with Val Accuracy: 75.30%
Epoch 5/100 | Train Loss: 0.2139 | Train Acc: 94.53%
Validation Loss: 0.9062 | Validation Acc: 75.30% | Sparsity: 90.55%

Epoch 6/100 | Train Loss: 0.1592 | Train Acc: 96.46%
Validation Loss: 0.9210 | Validation Acc: 75.16% | Sparsity: 90.55%

Epoch 7/100 | Train Loss: 0.1151 

In [77]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain without KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain without KD): 73.94%


In [78]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [79]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.91)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 39s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [80]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 6.9485 | Train Acc: 71.16% | Val Loss: 1.2721 | Val Acc: 69.37% | Sparsity: 90.55%
Epoch 2/50 | Train Loss: 3.7765 | Train Acc: 82.34% | Val Loss: 1.1684 | Val Acc: 72.38% | Sparsity: 90.55%
Epoch 3/50 | Train Loss: 2.6318 | Train Acc: 87.53% | Val Loss: 1.0641 | Val Acc: 74.48% | Sparsity: 90.55%
Epoch 4/50 | Train Loss: 2.0176 | Train Acc: 90.25% | Val Loss: 0.9707 | Val Acc: 76.32% | Sparsity: 90.55%
Epoch 5/50 | Train Loss: 1.5963 | Train Acc: 92.18% | Val Loss: 0.9384 | Val Acc: 76.98% | Sparsity: 90.55%
Epoch 6/50 | Train Loss: 1.3288 | Train Acc: 93.34% | Val Loss: 0.8451 | Val Acc: 78.05% | Sparsity: 90.55%
Epoch 7/50 | Train Loss: 1.1701 | Train Acc: 94.14% | Val Loss: 0.8461 | Val Acc: 78.16% | Sparsity: 90.55%
Epoch 8/50 | Train Loss: 1.0552 | Train Acc: 94.58% | Val Loss: 0.8532 | Val Acc: 78.18% | Sparsity: 90.55%
Epoch 9/50 | Train Loss: 0.9872 | Train Acc: 94.80% | Val Loss: 0.8186 | Val Acc: 78.68% | Sparsity: 90.55%
Epoch 10/50 | Train Loss: 0.

In [81]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 77.29%


In [124]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [125]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.91)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 35s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [126]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 1.9187 | Train Acc: 79.16% | Val Loss: 0.9187 | Val Acc: 74.78% | Sparsity: 90.55%
Epoch 2/50 | Train Loss: 0.9471 | Train Acc: 88.69% | Val Loss: 0.8491 | Val Acc: 76.40% | Sparsity: 90.55%
Epoch 3/50 | Train Loss: 0.6930 | Train Acc: 91.92% | Val Loss: 0.8264 | Val Acc: 77.25% | Sparsity: 90.55%
Epoch 4/50 | Train Loss: 0.5672 | Train Acc: 93.78% | Val Loss: 0.8008 | Val Acc: 78.11% | Sparsity: 90.55%
Epoch 5/50 | Train Loss: 0.4962 | Train Acc: 94.59% | Val Loss: 0.7930 | Val Acc: 78.20% | Sparsity: 90.55%
Epoch 6/50 | Train Loss: 0.4445 | Train Acc: 95.10% | Val Loss: 0.7901 | Val Acc: 78.53% | Sparsity: 90.55%
Epoch 7/50 | Train Loss: 0.4066 | Train Acc: 95.68% | Val Loss: 0.7732 | Val Acc: 78.48% | Sparsity: 90.55%
Epoch 8/50 | Train Loss: 0.3785 | Train Acc: 95.88% | Val Loss: 0.7806 | Val Acc: 78.60% | Sparsity: 90.55%
Epoch 9/50 | Train Loss: 0.3626 | Train Acc: 95.97% | Val Loss: 0.7764 | Val Acc: 78.12% | Sparsity: 90.55%
Epoch 10/50 | Train Loss: 0.

In [127]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 77.40%


# 80% Sparsity

In [85]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [86]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.81)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 35s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [87]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 4.25%


In [88]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=100,  save_path='retrained_student_model_80.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 77.99%
Epoch 1/100 | Train Loss: 0.3732 | Train Acc: 91.09%
Validation Loss: 0.7697 | Validation Acc: 77.99% | Sparsity: 80.60%

New best model saved with Val Accuracy: 78.28%
Epoch 2/100 | Train Loss: 0.1882 | Train Acc: 95.16%
Validation Loss: 0.7627 | Validation Acc: 78.28% | Sparsity: 80.60%

Epoch 3/100 | Train Loss: 0.1164 | Train Acc: 97.46%
Validation Loss: 0.7809 | Validation Acc: 78.08% | Sparsity: 80.60%

Epoch 4/100 | Train Loss: 0.0695 | Train Acc: 98.86%
Validation Loss: 0.8046 | Validation Acc: 78.00% | Sparsity: 80.60%

New best model saved with Val Accuracy: 78.40%
Epoch 5/100 | Train Loss: 0.0403 | Train Acc: 99.60%
Validation Loss: 0.8154 | Validation Acc: 78.40% | Sparsity: 80.60%

Epoch 6/100 | Train Loss: 0.0242 | Train Acc: 99.86%
Validation Loss: 0.8343 | Validation Acc: 78.30% | Sparsity: 80.60%

Epoch 7/100 | Train Loss: 0.0169 | Train Acc: 99.93%
Validation Loss: 0.8493 | Validation Acc: 78.13% | Sparsity: 80.60%

Epoch

In [89]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain without KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain without KD): 77.61%


In [ ]:
# # Measure inference times
# pruned_student_inference_time = measure_inference_time(pruned_student, test_loader, device)
# print(f"Student Model Inference Time(After Pruning): {pruned_student_inference_time * 1000:.2f} ms per batch")

In [90]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [91]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.81)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 34s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [92]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 2.9960 | Train Acc: 86.69% | Val Loss: 1.0049 | Val Acc: 73.75% | Sparsity: 80.60%
Epoch 2/50 | Train Loss: 1.7579 | Train Acc: 91.62% | Val Loss: 0.9158 | Val Acc: 76.43% | Sparsity: 80.60%
Epoch 3/50 | Train Loss: 1.2022 | Train Acc: 93.95% | Val Loss: 0.7754 | Val Acc: 79.13% | Sparsity: 80.60%
Epoch 4/50 | Train Loss: 0.9645 | Train Acc: 94.92% | Val Loss: 0.7349 | Val Acc: 80.00% | Sparsity: 80.60%
Epoch 5/50 | Train Loss: 0.8309 | Train Acc: 95.33% | Val Loss: 0.7231 | Val Acc: 80.38% | Sparsity: 80.60%
Epoch 6/50 | Train Loss: 0.7535 | Train Acc: 95.50% | Val Loss: 0.7054 | Val Acc: 80.47% | Sparsity: 80.60%
Epoch 7/50 | Train Loss: 0.6964 | Train Acc: 95.62% | Val Loss: 0.7214 | Val Acc: 80.41% | Sparsity: 80.60%
Epoch 8/50 | Train Loss: 0.6589 | Train Acc: 95.92% | Val Loss: 0.7297 | Val Acc: 80.42% | Sparsity: 80.60%
Epoch 9/50 | Train Loss: 0.6257 | Train Acc: 95.77% | Val Loss: 0.6995 | Val Acc: 80.94% | Sparsity: 80.60%
Epoch 10/50 | Train Loss: 0.

In [93]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 79.54%


In [94]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [95]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.81)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 33s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [96]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.7143 | Train Acc: 92.15% | Val Loss: 0.7414 | Val Acc: 79.34% | Sparsity: 80.60%
Epoch 2/50 | Train Loss: 0.4061 | Train Acc: 95.40% | Val Loss: 0.7104 | Val Acc: 80.21% | Sparsity: 80.60%
Epoch 3/50 | Train Loss: 0.3367 | Train Acc: 96.07% | Val Loss: 0.7014 | Val Acc: 80.59% | Sparsity: 80.60%
Epoch 4/50 | Train Loss: 0.3030 | Train Acc: 96.46% | Val Loss: 0.6945 | Val Acc: 80.55% | Sparsity: 80.60%
Epoch 5/50 | Train Loss: 0.2805 | Train Acc: 96.61% | Val Loss: 0.6896 | Val Acc: 80.62% | Sparsity: 80.60%
Epoch 6/50 | Train Loss: 0.2643 | Train Acc: 96.83% | Val Loss: 0.6893 | Val Acc: 80.79% | Sparsity: 80.60%
Epoch 7/50 | Train Loss: 0.2529 | Train Acc: 96.94% | Val Loss: 0.6916 | Val Acc: 80.77% | Sparsity: 80.60%
Epoch 8/50 | Train Loss: 0.2420 | Train Acc: 96.93% | Val Loss: 0.6885 | Val Acc: 80.86% | Sparsity: 80.60%
Epoch 9/50 | Train Loss: 0.2361 | Train Acc: 97.05% | Val Loss: 0.6905 | Val Acc: 80.85% | Sparsity: 80.60%
Epoch 10/50 | Train Loss: 0.

In [97]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 79.44%


# 75% of Sparsity

In [98]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [99]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.75062)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 39s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [100]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 5.03%


In [101]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=100,  save_path='retrained_student_model_75.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 79.50%
Epoch 1/100 | Train Loss: 0.2447 | Train Acc: 94.12%
Validation Loss: 0.7187 | Validation Acc: 79.50% | Sparsity: 74.69%

New best model saved with Val Accuracy: 79.75%
Epoch 2/100 | Train Loss: 0.1243 | Train Acc: 97.03%
Validation Loss: 0.7251 | Validation Acc: 79.75% | Sparsity: 74.69%

Epoch 3/100 | Train Loss: 0.0690 | Train Acc: 98.76%
Validation Loss: 0.7488 | Validation Acc: 79.51% | Sparsity: 74.69%

Epoch 4/100 | Train Loss: 0.0380 | Train Acc: 99.62%
Validation Loss: 0.7638 | Validation Acc: 79.09% | Sparsity: 74.69%

Epoch 5/100 | Train Loss: 0.0226 | Train Acc: 99.88%
Validation Loss: 0.7743 | Validation Acc: 79.47% | Sparsity: 74.69%

Epoch 6/100 | Train Loss: 0.0146 | Train Acc: 99.95%
Validation Loss: 0.7903 | Validation Acc: 79.02% | Sparsity: 74.69%

Early stopping triggered at epoch 7. No improvement for 5 epochs.
Best Validation Accuracy: 79.75% | Best Model Saved at: retrained_student_model_75.pt
Retraining completed i

In [102]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 78.95%


In [121]:
# # Measure inference times
# pruned_student_inference_time = measure_inference_time(retrained_student, test_loader,)
# print(f" Retrained pruned Student Model Inference Time: {pruned_student_inference_time * 1000:.2f} ms per batch")

 Retrained pruned Student Model Inference Time: 1.70 ms per batch


# Retrain with KD

In [103]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [104]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.75062)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 38s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [105]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 1.6158 | Train Acc: 92.72% | Val Loss: 0.8132 | Val Acc: 77.76% | Sparsity: 74.69%
Epoch 2/50 | Train Loss: 1.0973 | Train Acc: 94.54% | Val Loss: 0.7590 | Val Acc: 78.87% | Sparsity: 74.69%
Epoch 3/50 | Train Loss: 0.8748 | Train Acc: 95.20% | Val Loss: 0.6990 | Val Acc: 80.46% | Sparsity: 74.69%
Epoch 4/50 | Train Loss: 0.7578 | Train Acc: 95.57% | Val Loss: 0.7058 | Val Acc: 80.62% | Sparsity: 74.69%
Epoch 5/50 | Train Loss: 0.6786 | Train Acc: 95.64% | Val Loss: 0.7001 | Val Acc: 80.57% | Sparsity: 74.69%
Epoch 6/50 | Train Loss: 0.6351 | Train Acc: 95.92% | Val Loss: 0.6910 | Val Acc: 80.79% | Sparsity: 74.69%
Epoch 7/50 | Train Loss: 0.6021 | Train Acc: 95.79% | Val Loss: 0.6937 | Val Acc: 80.64% | Sparsity: 74.69%
Epoch 8/50 | Train Loss: 0.5724 | Train Acc: 96.00% | Val Loss: 0.6989 | Val Acc: 80.71% | Sparsity: 74.69%
Epoch 9/50 | Train Loss: 0.5517 | Train Acc: 95.89% | Val Loss: 0.6941 | Val Acc: 80.98% | Sparsity: 74.69%
Epoch 10/50 | Train Loss: 0.

In [106]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 80.14%


In [126]:
# # Measure inference times
# pruned_student_inference_time = measure_inference_time(pruned_student, test_loader, device)
# print(f" Retrained pruned Student Model Inference Time: {pruned_student_inference_time * 1000:.2f} ms per batch")

 Retrained pruned Student Model Inference Time: 2.00 ms per batch


In [107]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [108]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.75062)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 37s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [109]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.4798 | Train Acc: 94.64% | Val Loss: 0.6811 | Val Acc: 80.65% | Sparsity: 74.69%
Epoch 2/50 | Train Loss: 0.3085 | Train Acc: 96.16% | Val Loss: 0.6655 | Val Acc: 81.34% | Sparsity: 74.69%
Epoch 3/50 | Train Loss: 0.2684 | Train Acc: 96.77% | Val Loss: 0.6668 | Val Acc: 81.05% | Sparsity: 74.69%
Epoch 4/50 | Train Loss: 0.2484 | Train Acc: 96.84% | Val Loss: 0.6574 | Val Acc: 81.38% | Sparsity: 74.69%
Epoch 5/50 | Train Loss: 0.2320 | Train Acc: 97.03% | Val Loss: 0.6585 | Val Acc: 81.42% | Sparsity: 74.69%
Epoch 6/50 | Train Loss: 0.2246 | Train Acc: 97.10% | Val Loss: 0.6595 | Val Acc: 81.31% | Sparsity: 74.69%
Epoch 7/50 | Train Loss: 0.2153 | Train Acc: 97.17% | Val Loss: 0.6613 | Val Acc: 81.30% | Sparsity: 74.69%
Epoch 8/50 | Train Loss: 0.2091 | Train Acc: 97.15% | Val Loss: 0.6569 | Val Acc: 81.38% | Sparsity: 74.69%
Epoch 9/50 | Train Loss: 0.2035 | Train Acc: 97.29% | Val Loss: 0.6612 | Val Acc: 81.27% | Sparsity: 74.69%
Epoch 10/50 | Train Loss: 0.

In [110]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 79.78%


## 50% Sparsity

In [111]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [112]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.51)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 38s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [113]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 71.26%


In [114]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=200,  save_path='retrained_student_model_50%.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 81.03%
Epoch 1/200 | Train Loss: 0.1073 | Train Acc: 96.97%
Validation Loss: 0.6556 | Validation Acc: 81.03% | Sparsity: 50.75%

New best model saved with Val Accuracy: 81.41%
Epoch 2/200 | Train Loss: 0.0475 | Train Acc: 99.02%
Validation Loss: 0.6619 | Validation Acc: 81.41% | Sparsity: 50.75%

Epoch 3/200 | Train Loss: 0.0217 | Train Acc: 99.84%
Validation Loss: 0.6825 | Validation Acc: 81.26% | Sparsity: 50.75%

New best model saved with Val Accuracy: 81.43%
Epoch 4/200 | Train Loss: 0.0130 | Train Acc: 99.94%
Validation Loss: 0.6862 | Validation Acc: 81.43% | Sparsity: 50.75%

New best model saved with Val Accuracy: 81.55%
Epoch 5/200 | Train Loss: 0.0091 | Train Acc: 99.96%
Validation Loss: 0.6961 | Validation Acc: 81.55% | Sparsity: 50.75%

Epoch 6/200 | Train Loss: 0.0069 | Train Acc: 99.97%
Validation Loss: 0.7013 | Validation Acc: 81.42% | Sparsity: 50.75%

Epoch 7/200 | Train Loss: 0.0056 | Train Acc: 99.97%
Validation Loss: 0.7093 | V

In [115]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 80.69%


In [116]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [117]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.51)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 37s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [118]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.6105 | Train Acc: 96.69% | Val Loss: 0.6596 | Val Acc: 81.30% | Sparsity: 50.75%
Epoch 2/50 | Train Loss: 0.5517 | Train Acc: 96.22% | Val Loss: 0.6664 | Val Acc: 81.25% | Sparsity: 50.75%
Epoch 3/50 | Train Loss: 0.5220 | Train Acc: 96.14% | Val Loss: 0.6545 | Val Acc: 81.71% | Sparsity: 50.75%
Epoch 4/50 | Train Loss: 0.5009 | Train Acc: 96.13% | Val Loss: 0.6610 | Val Acc: 81.18% | Sparsity: 50.75%
Epoch 5/50 | Train Loss: 0.4873 | Train Acc: 96.13% | Val Loss: 0.6548 | Val Acc: 81.77% | Sparsity: 50.75%
Epoch 6/50 | Train Loss: 0.4668 | Train Acc: 96.20% | Val Loss: 0.6565 | Val Acc: 81.58% | Sparsity: 50.75%
Epoch 7/50 | Train Loss: 0.4573 | Train Acc: 96.23% | Val Loss: 0.6532 | Val Acc: 81.57% | Sparsity: 50.75%
Epoch 8/50 | Train Loss: 0.4471 | Train Acc: 96.17% | Val Loss: 0.6647 | Val Acc: 81.44% | Sparsity: 50.75%
Epoch 9/50 | Train Loss: 0.4367 | Train Acc: 96.23% | Val Loss: 0.6541 | Val Acc: 81.37% | Sparsity: 50.75%
Epoch 10/50 | Train Loss: 0.

In [119]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 80.99%


In [120]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [121]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.51)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 33s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [122]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.2154 | Train Acc: 96.99% | Val Loss: 0.6328 | Val Acc: 82.10% | Sparsity: 50.75%
Epoch 2/50 | Train Loss: 0.1839 | Train Acc: 97.30% | Val Loss: 0.6256 | Val Acc: 82.09% | Sparsity: 50.75%
Epoch 3/50 | Train Loss: 0.1727 | Train Acc: 97.38% | Val Loss: 0.6269 | Val Acc: 82.09% | Sparsity: 50.75%
Epoch 4/50 | Train Loss: 0.1663 | Train Acc: 97.51% | Val Loss: 0.6323 | Val Acc: 82.06% | Sparsity: 50.75%
Epoch 5/50 | Train Loss: 0.1625 | Train Acc: 97.51% | Val Loss: 0.6283 | Val Acc: 82.18% | Sparsity: 50.75%
Epoch 6/50 | Train Loss: 0.1593 | Train Acc: 97.50% | Val Loss: 0.6304 | Val Acc: 81.95% | Sparsity: 50.75%
Epoch 7/50 | Train Loss: 0.1564 | Train Acc: 97.65% | Val Loss: 0.6323 | Val Acc: 82.17% | Sparsity: 50.75%
Epoch 8/50 | Train Loss: 0.1546 | Train Acc: 97.63% | Val Loss: 0.6372 | Val Acc: 82.00% | Sparsity: 50.75%
Epoch 9/50 | Train Loss: 0.1526 | Train Acc: 97.53% | Val Loss: 0.6365 | Val Acc: 82.15% | Sparsity: 50.75%
Epoch 10/50 | Train Loss: 0.

In [123]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 81.01%
